# Packages to load those are not there:-

## Note: Don't need to run these if already ran like once!

In [ ]:
!pip install github

In [ ]:
!pip install --upgrade pip setuptools wheel


In [ ]:
!pip install seaborn

In [ ]:
!pip install onnxruntime onnx

In [ ]:
!pip install gnomad --no-deps

In [ ]:
!pip install psycopg2-binary

In [ ]:
!pip install skl2onnx


# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

In [ ]:
import hail as hl
# import os
# import onnx

# Initialize Hail with optimized Spark settings for 64 cores & 196GB RAM
hl.init(
    tmp_dir="/tmp",
    log="/tmp/hail.log",
    default_reference="GRCh38",
    spark_conf={
        "spark.executor.memory": "180g",  # Use 180GB for Spark, leave ~16GB for OS
        "spark.driver.memory": "180g",
        "spark.executor.cores": "64",  # Utilize all 64 cores
        "spark.driver.cores": "64",
        "spark.local.dir": "/tmp",
        "spark.sql.files.maxPartitionBytes": "32m",  # Efficient partitioning
        "spark.sql.shuffle.partitions": "256",  # Increase parallel shuffle performance
        "spark.memory.fraction": "0.9",  # Use 90% of allocated memory
        "spark.memory.storageFraction": "0.7",  # Optimize caching
        "spark.dynamicAllocation.enabled": "true",
        "spark.dynamicAllocation.minExecutors": "1",
        "spark.dynamicAllocation.maxExecutors": "64",
        "spark.sql.execution.arrow.enabled": "true",  # Faster DataFrame processing
        "spark.driver.extraJavaOptions": "-Dorg.slf4j.simpleLogger.defaultLogLevel=error"
    }
)

In [ ]:
# import hail as hl
# hl.init(idempotent = True)


In [ ]:
import onnx

In [ ]:
import gnomad

In [ ]:
from gnomad.sample_qc.ancestry import apply_onnx_classification_model

In [ ]:
from gnomad.utils.filtering import filter_to_adj

In [ ]:
from gnomad.sample_qc.ancestry import assign_population_pcs

In [ ]:
# v3.1 PCA loadings.
gnomad_v3_loadings = "/home/ky134/gnomad.v4.0.pca_loadings.ht/gnomad.v4.0.pca_loadings.ht"

# v3.1 ONNX RF model.
gnomad_v3_onnx_rf = "/home/ky134/gnomad.v4.0.RF_fit.onnx"

# Pipeline Starting for single vcf/vcf.gz

In [ ]:
# # Import the vcf
# input_folder = "/data/irb/pediatrics/pro00094341ncd/Sequencing (NC-DEFINE)/NC-DEFINE Manuscript SVD Prospective/"

#mt = hl.import_vcf(r"/data/irb/pediatrics/pro00094341ncd/Sequencing_NC_DEFINE/Balint_10012_vcfs/chr1.vcf.gz", array_elements_required=False, reference_genome = "GRCh38", force_bgz=True)
mt = hl.import_vcf(r"file.vcf.gz", reference_genome = "GRCh38", force_bgz=True, array_elements_required=False)

############# USE THE Uncomment out the below code for the file all which was from genomics core that we got after merging all the chr files ###############
#mt = mt.annotate_rows(info=mt.info.drop("AS_QUAL"))

# Split multi-allelic sites
#mt = hl.split_multi_hts(mt)

In [ ]:
# Drop the 'info' column from the MatrixTable
#mt = mt.drop(mt.info)

# Confirm the column is removed
mt.describe()


In [ ]:
# Define the base path for your GCS directory
base_path = "output/temp.anno"

# Output paths in the same directory
test_mt_output_path = f"{base_path}/all.anno_ancestry.mt"
test_scores_output_path = f"{base_path}/all.anno_ancestry.scores.ht"
gnomad_v3_assignment_path = f"{base_path}/all.anno_ancestry.assignment.ht"

# Print paths to verify
print("MatrixTable Output Path:", test_mt_output_path)
print("Scores Output Path:", test_scores_output_path)
print("Assignment Output Path:", gnomad_v3_assignment_path)


In [ ]:
v3_num_pcs = 20
v3_min_prob = 0.55


In [ ]:
v3_loading_ht = hl.read_table(gnomad_v3_loadings)

In [ ]:
# with hl.hadoop_open(gnomad_v3_onnx_rf, "rb") as f:
#     v3_onx_fit = onnx.load(f)

In [ ]:
# Load the ONNX model
with open(gnomad_v3_onnx_rf, "rb") as f:
    v3_onx_fit = onnx.load(f)

In [ ]:
mt.describe()

In [ ]:
# Filter rows based on the v3_loading_ht table
mt = mt.filter_rows(hl.is_defined(v3_loading_ht[mt.row_key]))

In [ ]:
mt = filter_to_adj(mt)

In [ ]:
read_if_exists = True

# mt = mt.checkpoint(
#     test_mt_output_path, overwrite=not _read_if_exists=read_if_exists
# )

mt = mt.checkpoint(
    test_mt_output_path, overwrite=not read_if_exists, _read_if_exists=read_if_exists
)

In [ ]:
# Project new genotypes onto loadings.
v3_pcs_ht = hl.experimental.pc_project(
    mt.GT,
    v3_loading_ht.loadings,
    v3_loading_ht.pca_af,
)

# Checkpoint PC projection results.
v3_pcs_ht = v3_pcs_ht.checkpoint(
    test_scores_output_path,
    overwrite=not read_if_exists,
    _read_if_exists=read_if_exists,
)

In [ ]:
# Assign populations using the ONNX model with max 20 PCs
ht, model = assign_population_pcs(
    v3_pcs_ht,
    pc_cols=v3_pcs_ht.scores[:v3_num_pcs],  # Restrict to at most 20 PCs
    fit=v3_onx_fit,
    min_prob=v3_min_prob,
    apply_model_func=apply_onnx_classification_model,
)

ht = ht.checkpoint(
    gnomad_v3_assignment_path,
    overwrite=not read_if_exists,
    _read_if_exists=read_if_exists,
)

In [ ]:

ht.aggregate(hl.agg.counter(ht.pop))

In [ ]:
v3_pcs_ht.show()


In [ ]:
v3_pcs_ht.describe()


In [ ]:
ht.count()

In [ ]:
ht.show(ht.count())

# CODE FOR ANCESTRY PREDICTION of multiple files in a folder:-

In [ ]:
# import hail as hl
# import os
# import onnx

# Initialize Hail
#hl.init()

# Paths and configurations
input_folder = "input_folder"
output_base_path = "output_folder"
gnomad_v3_loadings = "/home/gnomad.v4.0.pca_loadings.ht/gnomad.v4.0.pca_loadings.ht"
gnomad_v3_onnx_rf = "/home/gnomad.v4.0.RF_fit.onnx"

v3_min_prob = 0.55
read_if_exists = True

# Load PCA loadings and ONNX model
v3_loading_ht = hl.read_table(gnomad_v3_loadings)
with open(gnomad_v3_onnx_rf, "rb") as f:
    v3_onx_fit = onnx.load(f)

# Process all VCF files in the folder
vcf_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".vcf.gz")]

for vcf_file in vcf_files:
    print(f"Processing file: {vcf_file}")
    
    # Extract the base filename (e.g., 24291D-01-10 from 24291D-01-10.anno.vcf.gz)
    base_name = os.path.basename(vcf_file).replace(".vcf.gz", "")
    
    # Define output paths
    mt_output_path = f"{output_base_path}/{base_name}_ancestry.mt"
    scores_output_path = f"{output_base_path}/{base_name}_ancestry.scores.ht"
    assignment_output_path = f"{output_base_path}/{base_name}_ancestry.assignment.ht"
    
    # Import the VCF file
    mt = hl.import_vcf(vcf_file, reference_genome="GRCh38", array_elements_required=False, force_bgz=True)
    
    # Split multi-allelic sites
    # mt = hl.split_multi_hts(mt)
    
    # Filter rows based on PCA loadings table
    mt = mt.filter_rows(hl.is_defined(v3_loading_ht[mt.row_key]))
    
    # Apply additional filters if needed (e.g., filter to adj genotypes)
    mt = filter_to_adj(mt)
    
    # Save the processed MatrixTable
    mt = mt.checkpoint(mt_output_path, overwrite=not read_if_exists, _read_if_exists=read_if_exists)
    
    # Project new genotypes onto PCA loadings
    v3_pcs_ht = hl.experimental.pc_project(mt.GT, v3_loading_ht.loadings, v3_loading_ht.pca_af)
    
    # Save PC projection results
    v3_pcs_ht = v3_pcs_ht.checkpoint(scores_output_path, overwrite=not read_if_exists, _read_if_exists=read_if_exists)

    # Determine the number of available PCs but limit to 20
    # num_pcs = min(v3_pcs_ht.aggregate(hl.agg.max(hl.len(v3_pcs_ht.scores))), 20)
    num_pcs = 20
    
    # Assign populations using the ONNX model with max 20 PCs
    ht, model = assign_population_pcs(
        v3_pcs_ht,
        pc_cols=v3_pcs_ht.scores[:num_pcs],  # Restrict to at most 20 PCs
        fit=v3_onx_fit,
        min_prob=v3_min_prob,
        apply_model_func=apply_onnx_classification_model,
    )

    
    # Save the assignment results
    ht = ht.checkpoint(assignment_output_path, overwrite=not read_if_exists, _read_if_exists=read_if_exists)
    
    # Display population assignment results
    pop_counts = ht.aggregate(hl.agg.counter(ht.pop))
    print(f"Population assignment for {base_name}:")
    print(pop_counts)
    
    print(f"Completed processing for: {vcf_file}\n")
